# DerivaML Dataset Example.

DerivaML is a class library built on the Deriva Scientific Asset management system that is designed to help simplify a number of the basic operations associated with building and testing ML libraries based on common toolkits such as TensorFlow.  This notebook reviews the basic features of the DerivaML library.

## Set up DerivaML  for test case

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from deriva_ml.schema_setup.test_catalog import create_test_catalog, DemoML
from deriva_ml.deriva_ml_base import MLVocab as vt
from deriva_ml.dataset_bag import DatasetBag
import pandas as pd
from IPython.display import display, Markdown, HTML

Set the details for the catalog we want and authenticate to the server if needed.

In [22]:
hostname = 'dev.eye-ai.org'
domain_schema = 'demo-schema'

gnl = GlobusNativeLogin(host=hostname)
if gnl.is_logged_in([hostname]):
    print("You are already logged in.")
else:
    gnl.login([hostname], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")


You are already logged in.


Create a test catalog and get an instance of the DemoML class.

In [23]:
test_catalog = create_test_catalog(hostname, domain_schema)
ml_instance = DemoML(hostname, test_catalog.catalog_id)

## Configure DerivaML Datasets

In Deriva-ML a dataset is used to aggregate instances of entities.  However, before we can create any datasets, we must configure 
Deriva-ML for the specifics of the datasets.  The first stp is we need to tell Deriva-ML what types of use defined objects can be associated with a dataset.  

Note that out of the box, Deriva-ML is configured to allow datasets to contained dataset (i.e. nested datasets), so we don't need to do anything for that specific configuration.

In [24]:
print(f"Current dataset element types: {[a.name for a in ml_instance.list_dataset_element_types()]}")
ml_instance.add_dataset_element_type("Subject")
ml_instance.add_dataset_element_type("Image")
print(f"New dataset element types {[a.name for a in ml_instance.list_dataset_element_types()]}")

Current dataset element types: ['Dataset']
New dataset element types ['Dataset', 'Subject', 'Image']


Now that we have configured our datasets, we need to identify the dataset types so we can distinguish between them.

In [25]:
# Create a new dataset
ml_instance.add_term(vt.dataset_type, "DemoSet", description="A test dataset")
ml_instance.add_term(vt.dataset_type, 'Partitioned', description="A partitioned dataset for ML training.")
ml_instance.add_term(vt.dataset_type, "Subject", description="A test dataset")
ml_instance.add_term(vt.dataset_type, "Image", description="A test dataset")
ml_instance.add_term(vt.dataset_type, "Training", description="Training dataset")
ml_instance.add_term(vt.dataset_type, "Testing", description="Training dataset")
ml_instance.add_term(vt.dataset_type, "Validation", description="Validation dataset")

ml_instance.list_vocabulary_terms(vt.dataset_type)

[VocabularyTerm(name='DemoSet', synonyms=[], id='ml-test:38J', uri='/id/38J', description='A test dataset', rid='38J'),
 VocabularyTerm(name='Partitioned', synonyms=[], id='ml-test:38M', uri='/id/38M', description='A partitioned dataset for ML training.', rid='38M'),
 VocabularyTerm(name='Subject', synonyms=[], id='ml-test:38P', uri='/id/38P', description='A test dataset', rid='38P'),
 VocabularyTerm(name='Image', synonyms=[], id='ml-test:38R', uri='/id/38R', description='A test dataset', rid='38R'),
 VocabularyTerm(name='Training', synonyms=[], id='ml-test:38T', uri='/id/38T', description='Training dataset', rid='38T'),
 VocabularyTerm(name='Testing', synonyms=[], id='ml-test:38W', uri='/id/38W', description='Training dataset', rid='38W'),
 VocabularyTerm(name='Validation', synonyms=[], id='ml-test:38Y', uri='/id/38Y', description='Validation dataset', rid='38Y')]

Now create datasets and populate with elements from the test catalogs.

In [26]:
system_columns = ['RCT', 'RMT', 'RCB', 'RMB']

subject_dataset = ml_instance.create_dataset(['DemoSet', 'Subject'], description="A subject dataset")
image_dataset = ml_instance.create_dataset(['DemoSet', 'Image'], description="A image training dataset")
datasets = pd.DataFrame(ml_instance.find_datasets()).drop(columns=system_columns)
display(
    Markdown('## Datasets'),
    datasets)

## Datasets

,RID,Description,Dataset_Type
0,390,A subject dataset,"[DemoSet, Subject]"
1,396,A image training dataset,"[DemoSet, Image]"


And now that we have defined some datasets, we can add elements of the appropriate type to them.  We can see what is in our new datasets by listing the dataset members.

In [27]:
dp = ml_instance.domain_path  # Each call returns a new path instance, so only call once...
subject_rids = [i['RID'] for i in dp.tables['Subject'].entities().fetch()]
image_rids = [i['RID'] for i in dp.tables['Image'].entities().fetch()]

ml_instance.add_dataset_members(dataset_rid=subject_dataset, members=subject_rids)
ml_instance.add_dataset_members(dataset_rid=image_dataset, members=image_rids)

# List the contents of our datasets, and let's not include columns like modify time.
display(
    Markdown('## Subject Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(subject_dataset)['Subject']).drop(columns=system_columns),
    Markdown('## Image Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(image_dataset)['Image']).drop(columns=system_columns))

## Subject Dataset

,RID,Name
0,31A,Thing1
1,31C,Thing2
2,31E,Thing3
3,31G,Thing4
4,31J,Thing5
5,31M,Thing6
6,31P,Thing7
7,31R,Thing8
8,31T,Thing9
9,31W,Thing10


## Image Dataset

,RID,URL,Filename,Description,Length,MD5,Name,Subject
0,32J,/hatrac/Image/c8ba5497f0bed8245399eac8b2742f0b...,test_31A.txt,A test image,31,c8ba5497f0bed8245399eac8b2742f0b,None,31A
1,32M,/hatrac/Image/031101a98091754d6d7e7886c6800e85...,test_31C.txt,A test image,32,031101a98091754d6d7e7886c6800e85,None,31C
2,32P,/hatrac/Image/1041fbf65f3653945825c9c300ce1fc6...,test_31E.txt,A test image,32,1041fbf65f3653945825c9c300ce1fc6,None,31E
3,32R,/hatrac/Image/44e5ca34b8c49ff3a8d351e915a71f5c...,test_31G.txt,A test image,31,44e5ca34b8c49ff3a8d351e915a71f5c,None,31G
4,32T,/hatrac/Image/22bdc2e670ade1aa4532b20c8e5f10e3...,test_31J.txt,A test image,31,22bdc2e670ade1aa4532b20c8e5f10e3,None,31J
5,32W,/hatrac/Image/19f621276f730d6c272a16b5fdce40b8...,test_31M.txt,A test image,32,19f621276f730d6c272a16b5fdce40b8,None,31M
6,32Y,/hatrac/Image/3fe0f2cad987b486d21b67ae30611d29...,test_31P.txt,A test image,32,3fe0f2cad987b486d21b67ae30611d29,None,31P
7,330,/hatrac/Image/cc9f764f37bb28bef9870ae7d1854016...,test_31R.txt,A test image,31,cc9f764f37bb28bef9870ae7d1854016,None,31R
8,332,/hatrac/Image/7a87fef0956bf6d50601c7013790d99e...,test_31T.txt,A test image,31,7a87fef0956bf6d50601c7013790d99e,None,31T
9,334,/hatrac/Image/e8ba8f8fc6e995f534729c4c3b8db18f...,test_31W.txt,A test image,32,e8ba8f8fc6e995f534729c4c3b8db18f,None,31W


## Create partitioned dataset

Now let's create some subsets of the original dataset based on subject level metadata. We are going to create the subsets based on the metadata values of the subjects. We will download the subject dataset and look at its metadata to figure out how to partition the original data. Since we are not going to look at the images, we use download_dataset_bag, rather than materialize_bag.

In [29]:
bag_path, bag_rid = ml_instance.download_dataset_bag(subject_dataset)
ml_instance.materialize_dataset_bag(subject_dataset)
dataset_bag = DatasetBag(bag_path)
print(f"Bag materialized to {bag_path}")

Bag materialized to /var/folders/0k/27qzm97x3t7g3j1m6ksf_9f40000gn/T/tmpryaaxgik/390_6ded84f5d12d230f3e46cb6f8f375c125d2aa815bd0b52ad7883e96e22c4b239/Dataset_390


The domain model has two object: Subject and Images where an Image is associated with a subject, but a subject can have multiple images associated with it.  Let's look at the subjects and partition into test and training datasets.

In [30]:
# Get information about the subjects.....
subject_df = dataset_bag.get_table_as_dataframe('Subject')[['RID', 'Name']]
image_df = dataset_bag.get_table_as_dataframe('Image')[['RID', 'Subject', 'URL']]
metadata_df = subject_df.join(image_df, lsuffix="_subject", rsuffix="_image")
display(metadata_df)

,RID_subject,Name,RID_image,Subject,URL
0,31A,Thing1,32J,31A,/hatrac/Image/c8ba5497f0bed8245399eac8b2742f0b...
1,31C,Thing2,32M,31C,/hatrac/Image/031101a98091754d6d7e7886c6800e85...
2,31E,Thing3,32P,31E,/hatrac/Image/1041fbf65f3653945825c9c300ce1fc6...
3,31G,Thing4,32R,31G,/hatrac/Image/44e5ca34b8c49ff3a8d351e915a71f5c...
4,31J,Thing5,32T,31J,/hatrac/Image/22bdc2e670ade1aa4532b20c8e5f10e3...
5,31M,Thing6,32W,31M,/hatrac/Image/19f621276f730d6c272a16b5fdce40b8...
6,31P,Thing7,32Y,31P,/hatrac/Image/3fe0f2cad987b486d21b67ae30611d29...
7,31R,Thing8,330,31R,/hatrac/Image/cc9f764f37bb28bef9870ae7d1854016...
8,31T,Thing9,332,31T,/hatrac/Image/7a87fef0956bf6d50601c7013790d99e...
9,31W,Thing10,334,31W,/hatrac/Image/e8ba8f8fc6e995f534729c4c3b8db18f...


For ths example, lets partition the data based on the name of the subject.  Of course in real examples, we would do a more complex analysis in deciding
what subset goes into each data set.

In [31]:
def thing_number(name: pd.Series) -> pd.Series:
    return name.map(lambda n: int(n.replace('Thing','')))

training_rids = metadata_df.loc[lambda df: thing_number(df['Name']) % 3 == 0]['RID_image'].tolist()
testing_rids =  metadata_df.loc[lambda df: thing_number(df['Name']) % 3 == 1]['RID_image'].tolist()
validation_rids = metadata_df.loc[lambda df: thing_number(df['Name']) % 3 == 2]['RID_image'].tolist()

print(f'Training images: {training_rids}')
print(f'Testing images: {testing_rids}')
print(f'Validation images: {validation_rids}')


Training images: ['32P', '32W', '332', '338', '33E', '33M']
Testing images: ['32J', '32R', '32Y', '334', '33A', '33G', '33P']
Validation images: ['32M', '32T', '330', '336', '33C', '33J', '33R']


Now that we know what we want in each dataset, lets create datasets for each of our partitioned elements along with a nested dataset to track the entire collection.

In [32]:
nested_dataset = ml_instance.create_dataset(['Partitioned', 'Image'], description='A nested dataset for machine learning')
training_dataset = ml_instance.create_dataset('Training', description='An image dataset for training')
testing_dataset = ml_instance.create_dataset('Testing', description='A image dataset for testing')
validation_dataset = ml_instance.create_dataset('Validation', description='A image dataset for validation')
pd.DataFrame(ml_instance.find_datasets()).drop(columns=system_columns)

,RID,Description,Dataset_Type
0,390,A subject dataset,"[DemoSet, Subject]"
1,396,A image training dataset,"[DemoSet, Image]"
2,3BW,A nested dataset for machine learning,"[Partitioned, Image]"
3,3C2,An image dataset for training,[Training]
4,3C6,A image dataset for testing,[Testing]
5,3CA,A image dataset for validation,[Validation]


And then fill the datasets with the appropriate members.

In [13]:

ml_instance.add_dataset_members(dataset_rid=nested_dataset, members=[training_dataset, testing_dataset, validation_dataset])
ml_instance.add_dataset_members(dataset_rid=training_dataset, members=training_rids)
ml_instance.add_dataset_members(dataset_rid=testing_dataset, members=testing_rids)
ml_instance.add_dataset_members(dataset_rid=validation_dataset, members=validation_rids)


'3CA'

Ok, lets see what we have now.

In [14]:
display(
    Markdown('## Nested Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(nested_dataset)['Dataset']).drop(columns=system_columns),
    Markdown('## Training Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(training_dataset)['Image']).drop(columns=system_columns),
    Markdown('## Testing Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(testing_dataset)['Image']).drop(columns=system_columns),
    Markdown('## Validation Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(validation_dataset)['Image']).drop(columns=system_columns))

## Nested Dataset

,RID,Description
0,3BW,A nested dataset for machine learning


## Training Dataset

,RID,URL,Filename,Description,Length,MD5,Name,Subject
0,32P,/hatrac/Image/db94fb68cdd71445540cc7fa27a07d4f...,test_31E.txt,A test image,31,db94fb68cdd71445540cc7fa27a07d4f,None,31E
1,32W,/hatrac/Image/6e24ac002de2bdd782690a3db2b3faad...,test_31M.txt,A test image,31,6e24ac002de2bdd782690a3db2b3faad,None,31M
2,332,/hatrac/Image/4933c2542c5eebcccc8ff977c18d988f...,test_31T.txt,A test image,32,4933c2542c5eebcccc8ff977c18d988f,None,31T
3,338,/hatrac/Image/a57a5545080ec09961b50db20f9a6594...,test_320.txt,A test image,31,a57a5545080ec09961b50db20f9a6594,None,320
4,33E,/hatrac/Image/b6d8dfe650289aec63f607d0b8d48f6d...,test_326.txt,A test image,32,b6d8dfe650289aec63f607d0b8d48f6d,None,326
5,33M,/hatrac/Image/ce947fa75efb8a6ca97c1b3582e37abd...,test_32C.txt,A test image,32,ce947fa75efb8a6ca97c1b3582e37abd,None,32C


## Testing Dataset

,RID,URL,Filename,Description,Length,MD5,Name,Subject
0,32J,/hatrac/Image/e8d46b11afbcc4c9fed87844abec9639...,test_31A.txt,A test image,31,e8d46b11afbcc4c9fed87844abec9639,None,31A
1,32R,/hatrac/Image/ba1ca03327eee1da6dabf6e2ad4cdb6a...,test_31G.txt,A test image,31,ba1ca03327eee1da6dabf6e2ad4cdb6a,None,31G
2,32Y,/hatrac/Image/f57883ebb4509d1de3c64ff38be28933...,test_31P.txt,A test image,31,f57883ebb4509d1de3c64ff38be28933,None,31P
3,334,/hatrac/Image/7ca4ef4169df48aaa53141c5657c6dd5...,test_31W.txt,A test image,31,7ca4ef4169df48aaa53141c5657c6dd5,None,31W
4,33A,/hatrac/Image/cfaa7156ff6188dd86ea9a775fb24d27...,test_322.txt,A test image,30,cfaa7156ff6188dd86ea9a775fb24d27,None,322
5,33G,/hatrac/Image/3591fdd83df3e52039e74806d9d43588...,test_328.txt,A test image,31,3591fdd83df3e52039e74806d9d43588,None,328
6,33P,/hatrac/Image/1b4d9a2f0d6009b295e47959497a8190...,test_32E.txt,A test image,31,1b4d9a2f0d6009b295e47959497a8190,None,32E


## Validation Dataset

,RID,URL,Filename,Description,Length,MD5,Name,Subject
0,32M,/hatrac/Image/ffb678d1d2d9d174d0a5a46172e50b7f...,test_31C.txt,A test image,30,ffb678d1d2d9d174d0a5a46172e50b7f,None,31C
1,32T,/hatrac/Image/a48b838c9d2709e1a7afa88ef15d4fa4...,test_31J.txt,A test image,31,a48b838c9d2709e1a7afa88ef15d4fa4,None,31J
2,330,/hatrac/Image/13f4fcfc96fe84dfc40cf1a836d12393...,test_31R.txt,A test image,31,13f4fcfc96fe84dfc40cf1a836d12393,None,31R
3,336,/hatrac/Image/de63214e4319c1ddd4ecb875509c2fcb...,test_31Y.txt,A test image,31,de63214e4319c1ddd4ecb875509c2fcb,None,31Y
4,33C,/hatrac/Image/5fd608afbaadf23ecfa6ce62a608ca63...,test_324.txt,A test image,31,5fd608afbaadf23ecfa6ce62a608ca63,None,324
5,33J,/hatrac/Image/e696028ffb9be3cc477fb32656f218a9...,test_32A.txt,A test image,31,e696028ffb9be3cc477fb32656f218a9,None,32A
6,33R,/hatrac/Image/794251f097b71a879db91decbd0ff59b...,test_32G.txt,A test image,31,794251f097b71a879db91decbd0ff59b,None,32G


As our very last step, lets get a PID that will allow us to share and cite the dataset that we just created

In [15]:
dataset_citation = ml_instance.cite(nested_dataset)
display(
    HTML(f'Nested dataset citation: <a href={nested_dataset}>{nested_dataset}</a>')
)

In [16]:
display(HTML(f'<a href={ml_instance.chaise_url("Dataset")}>Browse Datasets</a>'))

In [17]:
test_catalog.delete_ermrest_catalog(really=True)

<Response [204]>